# Selección de Fenotipos para finetuning de BioBERT
Domingo Méndez García. [domingo.mendezg@um.es](mailto:domingo.mendezg@um.es) [github.com/user/DgoMndez](https://github.com/DgoMndez)
* Referencias:
  * Modelo de partida: [pritamdeka](https://huggingface.co/pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb)
  * Ontología: HPO versión https://github.com/obophenotype/human-phenotype-ontology/releases/tag/v2022-12-15

TODO:
* Breve sobre el problema y modelo que estamos tratando.
* Resultados del finetuning anterior: ¿por qué cambiar los fenotipos?
* Análisis de la ontología.
  * IC y profundidad: las variables a tener en cuenta.
  * Método de selección.
  * Resultados.
  * Decisión final informada.
  
## Problema: representación de fenotipos de HPO

El modelo BERT es un Sentence-Transformer que mapea textos a vectores de 768 componentes que llamamos "embeddings", y puede adaptarse a diferentes tareas. Está especializado en textos científicos y médicos pero queremos fine-tunearlo usando un corpus de abstracts de PUBMED para mejorar su desempeño como etapa en PhenoLinker, un sistema que infiere relaciones entre genes y fenotipos para predecir patogenicidad. Los fenotipos que consideramos son los de la subontología Phenotypic Abnormality de HPO (que abreviamos HPO:PA). El objetivo entonces es conseguir que el embedding del nombre de un fenotipo represente mejor al fenotipo como nodo de la ontología. Para medir esto comparamos la similitud de los fenotipos en HPO (Lin) con la similitud coseno entre embeddings.

## Evaluación del experimento de finetuning

Notebook del experimento en [results-lprogress.ipynb](https://github.com/DgoMndez/DL-patogen-colab-DIIC/blob/main/src/fine-tuning/evaluation/results-lprogress.ipynb), con un resumen de cómo se ha entrenado al principio.

### Datos:
* **Ejemplos de entrenamiento:** pares (abstract, fenotipo).
* **Ejemplos de evaluación:** (fenotipo1, fenotipo2, gold), donde gold es la similitud Lin entre los fenotipos y se compara con la similitud coseno entre embeddings.

En concreto:

* Muestra fenotipos/etiquetas: [index.csv](../../data/phenotypes/index.csv)
  * $M = 100$ tags o fenotipos de entrenamiento. 
  * Todos los pares de tags fueron usados para la evaluación de training.
* Abstracts de los que se tomó la muestra: [abstracts.csv](../../data/abstracts/abstracts.csv)
  * $N = 11613$ abstracts.
  * Obtenidos de una búsqueda en [pubmed](https://pubmed.ncbi.nlm.nih.gov/)
* Fenotipos test: [leaf-phenotypes.csv](../../data/phenotypes/leaf-phenotypes.csv) = nodos hoja HPO:PhenotypicAbnormality.
  * Muestra de 1000 pares de fenotipos para la evaluación de test [pairs-test.csv](../data/evaluation/pairs-test.csv).
## Resumen del modelo:
Modelo original: [pritamdeka](https://huggingface.co/pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb).  
Modelo obtenido: [README.md](../../output/fine-tuned-bio-bert-ev-mse-01-04-2024/README.md)  

* Medidas:

![Pearson correlation vs Batches](./figures/pearson.png)

![Spearman correlation vs Batches](./figures/spearman.png)

![MSE vs Batches](./figures/spearman.png)

* **Conclusiones**: Los mejores scores de evaluación se alcanzan a los pocos batches. Este finetuning no ha funcionado bien porque se alcanza el límite de aprendizaje muy pronto, pero no se percibe sobreajuste porque la tendencia del score train y test se parece.

* **Justificación**:
  * Distintas funciones de pérdida (BatchAllTripletLoss, CosineSimilarityLoss) o distintos hiperparámetros (lr, weight_decay, margin) pueden dar mejores resultados pero no creo que sea el factor determinante.
  * La selección de fenotipos y el tamaño del índice de etiquetas es el factor determinante. Los nodos hoja no están bien representados en la ontología (casi todos tienen similitud lin ~ 0 seguramente porque no aparecen frecuentemente en la BD usada para calcular las similitudes). Esta es la principal explicación de los malos resultados del experimento: estos fenotipos hoja no son útiles para la evaluación y no representan bien HPO. Consecuentemente, hay que volver a obtener un corpus de abstracts con las búsquedas de los nuevos fenotipos y volver a preparar nuevos pares de evaluación y test.

![Lin histogram](./figures/lin.png)

Como vemos la gran mayoría de pares de fenotipos tanto de evaluación como de test tienen similitud 0. La similitud Lin se calcula a partir del IC de cada uno de los fenotipos y del de su ancestro común más profundo.

![IC distribution](./figures/ic-dist-0.png) 

La distribución del IC de los fenotipos es bimodal por la gran cantidad de fenotipos con IC=0 ("nulos"). Estos fenotipos estimamos que causan problemas porque: la medida de similitud no es fiable (no hay ejemplos en la ontología para calcular el IC) y es muy probable que no se encuentren suficientes papers en PUBMED sobre ellos.

"The information content of each node in the HPO can be estimated through its frequency among annotations of the entire OMIM corpus." - [The Human Phenotype Ontology: A Tool for Annotating and Analyzing Human Hereditary Disease](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2668030/)

## Análisis de la ontología

A partir de los resultados anteriores queremos seleccionar un conjunto de fenotipos etiqueta que represente HPO:PA y nos sirva para entrenar, que llamaremos índice. El primer índice de fenotipos era una muestra de tamaño 100 de los nodos hoja de HPO:PA y no funcionó como deseábamos. Para obtener un mejor conjunto de etiquetas (fenotipos) hemos analizado HPO:PA para tomar una decisión.

### IC y profundidad

<figure>
  <img src="./figures/ic-hist-full.png" alt="IC-hist-full">
  <figcaption>Histograma de IC para HPO:PA completa</figcaption>
</figure>

El histograma de IC de HPO:PA muestra la gran cantidad de fenotipos nulos que hay.

<figure>
  <img src="./figures/meandepth.png" alt="meandepth">
  <figcaption>Profundidad media vs Porcentaje de nodos</figcaption>
</figure>

En el eje X tenemos porcentajes de muestra. La profundidad media en Y quiere decir la profundidad media del X% de nodos menos profundos de la ontología. Esta idea de selección fue descartada por la siguiente.

Tanto el IC como la profundidad son estimadores de la especifidad de un término de HPO. La profundidad se basa únicamente en la estructura de la ontología mientras que el IC se basa en la frecuencia del término (y sus hijos) en el corpus de referencia para la ontología. Por eso usaremos el IC medio de un índice como medida de bondad. Para seleccionar el índice "cortamos" el árbol a cierta profundidad $d$ y nos quedamos con los nodos hoja de ese subárbol:
<a name="selection-method"></a>
* Escogemos una profundidad $d$.
* Seleccionamos todos los nodos hoja a profundidad menor que $d$.
* Seleccionamos todos los nodos a profundidad $d$.
  * Quitamos todos los nodos nulos (IC=0). Después del análisis añadimos este paso.
* El índice será una muestra de tamaño 2000 de los fenotipos que quedan. El tamaño viene determinado por la capacidad de búsqueda de abstracts en PUBMED y la potencia de cálculo para el finetuning. Antes habíamos entrenado con un tamaño 100 de índice y una CPU, tardando 8h, ahora hemos tomado un índice 20 veces mayor esperando que con GPU tengamos un finetuning mucho más rápido.

<figure>
  <img src="./figures/tags-vs-depth.png" alt="tags_vs_depth">
  <figcaption>Número de etiquetas del índice vs Profundidad</figcaption>
</figure>

La gráfica muestra el número de etiquetas de índice del corte (contando los nulos), lo que nos dio la primera idea del tamaño de muestra a cada profundidad.

### Resultados del análisis
Para realizar el análisis se tomaron varias medidas a cada profundidad con el objetivo de escoger la más apropiada.

1. **Contar**

Lo primero fue contar el número de nodos (count), nodos hoja (leafs), nodos nulos (zeros), media y cuasivarianza de IC (mean, var) y tamaño de la selección (chosen) a cada profundidad. En este primer conteo no se habían eliminado los nulos de la selección.

In [1]:
import pandas as pd
dfDepth = pd.read_csv('results/depth_count.csv', sep='\t')
columns = ['depth', 'count', 'leafs', 'zeros', 'chosen', 'mean', 'var']
display(dfDepth[columns])

,depth,count,leafs,zeros,chosen,mean,var
0,0,1,0,0.0,1,0.000817,NaN
1,1,23,0,0.0,23,1.177510,0.920597
2,2,155,28,17.0,155,3.186360,5.403676
3,3,800,318,138.0,828,4.368461,8.009591
4,4,2157,1198,535.0,2503,4.306913,9.162556
5,5,3789,2502,1158.0,5333,4.230784,10.081808
6,6,3696,2569,1227.0,7742,4.321174,11.109246
7,7,2886,1985,1217.0,9501,3.920541,12.640566
8,8,1870,1535,1077.0,10470,2.890284,12.384680
9,9,678,543,330.0,10813,3.428010,12.456000


Lo que vi fue que el IC medio aumentaba hasta profundidad 6 y después disminuía, y por la alta varianza especialmente a profundidades mayores me di cuenta de que los nulos y el carácter bimodal del IC tenían mucho que ver. Al principio pensaba que solo había tantos nulos a profundidades mayores, que son las que pensaba que tenían nodos más específicos. Al ver que había muchos nodos nulos en todas las profundidades decidí quitarlos y repetir los análisis de IC medio y varianza sin ellos.

<figure>
  <img src="./results/images/ic_depth_5.png" alt="hist d=5">
  <figcaption>Histograma de IC a profundidad 5</figcaption>
</figure>

En cualquier histograma a cualquier profundidad notamos una gran cantidad de nulos.

2. **Nulos vs no-nulos**

Ahora repetía el conteo midiendo por separado el IC medio (meanGTZ) y varianza (varGTZ) de los no nulos a cada profundidad.

In [4]:
columns = ['depth', 'count', 'chosen', 'zeros', 'notZeros', 'meanGTZ', 'varGTZ', 'mean', 'var']
display(dfDepth[columns])

,depth,count,chosen,zeros,notZeros,meanGTZ,varGTZ,mean,var
0,0,1,1,0.0,1.0,0.000817,NaN,0.000817,NaN
1,1,23,23,0.0,23.0,1.177510,0.920597,1.177510,0.920597
2,2,155,155,17.0,138.0,3.578882,4.659159,3.186360,5.403676
3,3,800,828,138.0,662.0,5.279107,4.867119,4.368461,8.009591
4,4,2157,2503,535.0,1622.0,5.727504,4.045136,4.306913,9.162556
5,5,3789,5333,1158.0,2631.0,6.092907,3.170809,4.230784,10.081808
6,6,3696,7742,1227.0,2469.0,6.468634,2.735594,4.321174,11.109246
7,7,2886,9501,1217.0,1669.0,6.779318,2.471154,3.920541,12.640566
8,8,1870,10470,1077.0,793.0,6.815676,2.437992,2.890284,12.384680
9,9,678,10813,330.0,348.0,6.678709,2.528724,3.428010,12.456000


Lo que vi fue que la varianza se reducía considerablemente y ahora el IC medio era creciente con la profundidad (salvo profundidades 13 y 14 que solo tienen nulos). Así que a partir de aquí la hipótesis era utilizar una profundidad de 10, que es muy alta y a partir de ahí se añaden muy pocos no nulos.

<figure>
  <img src="./results/images/ic_depth_10_not_zeros.png" alt="hist IC>0 d=10">
  <figcaption>Histograma de IC>0 a profundidad 10</figcaption>
</figure>

Ahora al  quitar los nulos en cualquier profundidad (por ejemplo 10) vemos que predominan los nodos con mayor IC en la selección y esperamos que esto signifique una similitud Lin más fiable entre pares de fenotipos.

Pero antes de tomar la decisión final debía repetir los conteos en lugar de considerando la profundidad $d$ justa, considerando la selección (añadiendo en cada fila los nodos hoja de profundidad $<d$). También debía añadir el % cubierto de ontología con una muestra de tamaño 2000, para garantizar que la muestra es suficientemente grande como para representar la ontología.

3. **Selección y muestra final**

Se realizaron los conteos y medidas del IC para la selección especificada en [IC y profundidad](#ic-y-profundidad) y finalmente se calculó para cada profundiad el % de ontología cubierto por una muestra aleatoria de tamaño 2000.

El % de ontología cubierto $cub(F)$ de un conjunto de fenotipos $F$ se define de la siguiente manera:
* Tomar $S$ el conjunto de nodos hoja alcanzables desde algún nodo de $F$ por las aristas de relación padre-hijo de la ontología.
* Tomar $S^*$ el subconjunto de los nodos no nulos de $S$.
* $cub(F) = 100|S|/H^*\%$ donde $H^*$ es el conjunto de nodos hoja no nulos de HPO:PA.

Ahora trueCount, trueMean y trueVar son el número de nodos, la media de IC y la cuasivarianza para la selección especificada a profundidad $d$. sample  es el tamaño de muestra y sampleCoverPercent el % de ontología cubierto por la muestra.

<figure>
  <img src="./results/images/ic_depth_10_true.png" alt="hist d=10">
  <figcaption>Histograma de IC de la selección a profundidad 10</figcaption>
</figure>

La distribución del IC de la selección es similar a la de profundidad 10, los nodos con más información son más frecuentes, pero en este caso hemos considerado toda la selección de los 6107 nodos no nulos.

In [5]:
columns = ['depth', 'trueCount', 'trueMean', 'trueVar', 'sampleCoverPercent', 'sample']
display(dfDepth[columns])

,depth,trueCount,trueMean,trueVar,sampleCoverPercent,sample
0,0,1.0,0.000817,NaN,100.000000,1
1,1,23.0,1.177510,0.920597,100.000000,23
2,2,138.0,3.578882,4.659159,100.000000,138
3,3,680.0,5.308145,4.819274,100.000000,680
4,4,1846.0,5.876124,3.957335,100.000000,1846
5,5,3610.0,6.336812,2.997269,62.807075,2000
6,6,4942.0,6.688773,2.387720,43.039633,2000
7,7,5688.0,6.905436,2.055930,37.127416,2000
8,8,5916.0,6.978093,1.920782,34.261382,2000
9,9,6049.0,7.003977,1.859949,33.393384,2000


<figure>
  <img src="./figures/samplecover.png" alt="samplecover">
  <figcaption>% Cubierto de H* vs profundidad</figcaption>
</figure>

En la tabla vemos que se repite la tendencia creciente del IC con la profundidad y que el % cubierto de la ontología decrece con la profundidad hasta el 32.75%. Con esto se toma la decisión final de a qué profundidad tomar la selección y si la muestra de tamaño 2000 es válida.

**Decisión**: tomar la selección (sin nulos) a profundidad $d=10$ porque:
* El IC medio de la selección crece con la profundidad, pero se estanca a esa profundidad 10 en las centésimas, aparte de que a partir de esa profundidad se añaden muy pocos nodos no nulos.
* Con una muestra de tamaño 2000 de la selección a profundidad 10 cubrimos un 32.82% de los nodos hoja no nulos de la ontología, que es un porcentaje suficiente.
* Importante considerar que si hacemos la muestra completa de 6106 nodos no nulos cubrimos el 100%.
* La clave ha sido quitar los nulos, lo que ha dado unos valores de IC medio y sampleCoverPercent aceptables a partir de $d=5$.